## Predicting diseases  - Agregation vs Mitochondrial toxicity with PowerTransformer
### StratifiedKFold


In [ ]:
%load_ext autoreload
%autoreload 2

# Imports

In [ ]:
import pandas as pd
import numpy as np

from pathlib import Path 

import matplotlib.pyplot as plt
import seaborn as sns
import os

In [ ]:
from pprint import pprint

In [ ]:
from IPython.display import display_html

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [ ]:
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [ ]:
from scipy.stats import gaussian_kde

from numpy import  interp

from itertools import cycle

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix, classification_report, roc_auc_score, roc_curve, auc

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

In [ ]:
from tensorflow.random import set_seed

In [ ]:
set_seed(42)

In [ ]:
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
import pickle

In [ ]:
from sklearn.model_selection import StratifiedKFold, train_test_split

In [ ]:
pd.set_option("display.max_columns", 100)  

In [ ]:
from sklearn.preprocessing import PowerTransformer
from scipy import optimize
class SafePowerTransformer(PowerTransformer):
    def _yeo_johnson_optimize(self, x):
        def _neg_log_likelihood(lmbda):
            x_trans = self._yeo_johnson_transform(x, lmbda)
            n_samples = x.shape[0]
            variance = x_trans.var()
            if variance == 0:
                return np.inf  
            loglike = -n_samples / 2 * np.log(variance) 
            loglike += (lmbda - 1) * (np.sign(x) * np.log1p(np.abs(x))).sum()
            return -loglike
        x = x[~np.isnan(x)]
        return optimize.brent(_neg_log_likelihood, brack=(-2, 2))

In [ ]:
img_dpi= 300

# My package

In [ ]:
from neurocrick import plotter
from neurocrick.utils import display_side_by_side

In [ ]:
from sinfo import sinfo
sinfo()


# Functions

In [ ]:
def load_data(data_file):
    #loads the mito coloc cleaned data, excludes columns :
    #Unnamed: 1 , lysosome morphology Ratio Width to Length
    df = pd.read_csv(data_file, index_col=0)
    df = df.drop(columns=['Unnamed: 1'])
    # lysosome morphology length, width and ratio of width to length is there
    col_exclude= ['lysosome morphology Ratio Width to Length']
    df = df.drop(col_exclude, axis=1)
    df=df.reset_index()
    df = df.drop(columns=['index'])
    experiment_cols = ["PlateID"]
    all_feats = [col for col in df.columns if 'label' not in col and 'PlateID' not in col]
    print("Data loaded from ", data_file)

    return df, experiment_cols, all_feats

In [ ]:
#Make model aggregation vs mito_toxicity
#Powerscale PER CONTROL, WITHIN EACH PLATE SEPARATELY (ie. per control per plate)
#Fill the nans in each plate according to the other feature values IN THAT PLATE
#Use a power transformer to scale the features within each plate

def scale_trans_by_control_per_plate(df, feat_cols):
    imp = IterativeImputer(max_iter=10, random_state=0, initial_strategy='median')
    
    lambdas = {}

    fin_df = []
    for eachplate in df.PlateID.unique():
        dd = df[df.PlateID==eachplate].copy()

        #print(f"plate:{eachplate}, shape:{dd.shape}")

        # fit for scaling. Scale per control per plate, then apply the transformation to all datapoints in the plate.
        scaler = SafePowerTransformer()
        
        scaler.fit(dd[dd.label_name=='control'][feat_cols].values)

        # scale
        dd.loc[:,feat_cols] = scaler.transform(dd[feat_cols].values)

        # inpute missing values using iterative inputer
        dd.loc[:,feat_cols] = imp.fit_transform(dd[feat_cols].values)

        lambdas[eachplate] = scaler.lambdas_
        fin_df.append(dd)

    fin_df = pd.concat(fin_df)
    return fin_df, lambdas


In [ ]:
def scale_data_and_exclude_controls(df_train, df_val, df_test, all_feats, var_cutoff):
    #function scales the data and excludes features using the same 
    #cut-off as in the run without stratifiedKFold()  
    #it then excludes the controls from the datasets
    scal_df_train, lambdas_train = scale_trans_by_control_per_plate(df_train, all_feats)
    scal_df_train = scal_df_train.sample(frac = 1, random_state=200) #shuffle
    
    lambdas_train_df = pd.DataFrame(np.array(list(lambdas_train.values())), columns=all_feats, index=list(lambdas_train.keys()))
    lambdas_train_df.index.rename("Plate_ID", inplace=True)
    lambdas_var = lambdas_train_df.var().sort_values(ascending = False)
    
    # Exclude the features with a high variance in the lambdas in the training set
    #var_cutoff = 50 # to exclude the single one with the max variance
    col_2_exclude = lambdas_var[lambdas_var > var_cutoff].index
    scal_df_train = scal_df_train.drop(col_2_exclude, axis=1)
    
    # Scaling the test and validation set and exclude the same features as in the training set 
    #scaling
    scal_df_val, lambdas_val = scale_trans_by_control_per_plate(df_val, all_feats)
    scal_df_val = scal_df_val.sample(frac= 1, random_state = 200)
    scal_df_test, lambdas_test = scale_trans_by_control_per_plate(df_test, all_feats)
    scal_df_test = scal_df_test.sample(frac= 1, random_state = 200)

    # Dropping the same features from the val and test set as in the training set
    scal_df_val = scal_df_val.drop(col_2_exclude, axis=1)
    scal_df_test = scal_df_test.drop(col_2_exclude, axis=1)
    screen_feats = list(set(all_feats)-set(col_2_exclude))
    
    #Remove control from the test, train and val datasets
    control_code = (scal_df_train.loc[scal_df_train.label_name =="control"].label.unique())[0]
    scal_df_train = drop_observations_control(scal_df_train, control_code)
    scal_df_val = drop_observations_control(scal_df_val, control_code)
    scal_df_test = drop_observations_control(scal_df_test, control_code)
    
    return scal_df_train, scal_df_val, scal_df_test, screen_feats

In [ ]:
#Dropping the control as looking at diseases
def drop_observations_control(df, control_code):
    filter_dfs = df[df.label!=control_code].copy()

    filter_dfs['label'] = filter_dfs.label - min(filter_dfs.label.unique())

    filter_dfs.label_name = filter_dfs.label_name.cat.remove_unused_categories()

    filter_dfs = filter_dfs.reset_index(drop=True)
    
    return filter_dfs

In [ ]:
def evaluate_model(data, all_features, n_folds, variance_threshold, fname_prefix_save_models):
#data =df
#n_folds=5
#variance_threshold=50
#all_features = all_feats

    X = data[all_features]
    y = data.label

    scores, histories = list(), list()
    results = list()
    fold_ct = 0
    # prepare cross validation
    skf = StratifiedKFold(n_splits= n_folds, shuffle=True, random_state=1)
    for train_index, test_index in skf.split(X, y):
        fold_ct+=1
        print("Fold", fold_ct)
        df_train, df_test = data.iloc[train_index], data.iloc[test_index]
        

        df_train, df_val = train_test_split(df_train,   
                                          random_state=42, stratify=df_train.label)
        #scale and drop controls
        s_df_train, s_df_val, s_df_test, screen_features = scale_data_and_exclude_controls(df_train, df_val, df_test, all_features, variance_threshold )


        #del df_train, df_val, df_test
        X_train = s_df_train[screen_features].values
        X_val = s_df_val[screen_features].values
        X_test = s_df_test[screen_features].values

        y_train = s_df_train['label'].values
        y_val = s_df_val['label'].values
        y_test = s_df_test['label'].values

        #print("Train:", X_train.shape, " ", y_train.shape)
        #print("Val:", X_val.shape, " ", y_val.shape)
        #print("Test:", X_test.shape, " ", y_test.shape)

        y_test_onehot = to_categorical(y_test)
        y_val_onehot = to_categorical(y_val)
        y_train_onehot = to_categorical(y_train)

        # define model
        early_stop = tf.keras.callbacks.EarlyStopping(
            monitor='val_loss', 
            min_delta=0, 
            patience=50, 
            verbose=0, 
            mode='auto',
            baseline=None, 
            restore_best_weights=True,
        )
        num_dense = 25
        scal_model_name =  fname_prefix_save_models.parent.joinpath(os.path.basename(fname_prefix_save_models) + f"pTrans-control-plate-scaling_dense-{num_dense}_3-dropout_fold{fold_ct}")

        scal_checkpoint = ModelCheckpoint(
                str(scal_model_name)+'.hdf5', 
                monitor='val_loss', 
                verbose=0,
                save_best_only=True, 
                mode='auto', 
                save_freq='epoch'
            )
        dropout_rate = 0.2

        input_t = Input(shape=(X_train.shape[1],))

        x = Dense(num_dense, activation='relu')(input_t)
        x = Dropout(dropout_rate)(x)

        x = Dense(num_dense, activation='relu')(x)
        x = Dropout(dropout_rate)(x)

        x = Dense(num_dense, activation='relu')(x)
        x = Dropout(dropout_rate)(x)


        output = Dense(y_train_onehot.shape[1], activation='softmax')(x)

        scal_model = Model(inputs=input_t, outputs = output)
        scal_model.compile(
            loss='categorical_crossentropy',
            metrics=['accuracy'],
            optimizer='adam',
        )

        batch_size = 64
        epochs = 200 

        # fit model
        history = scal_model.fit(
            X_train,
            y_train_onehot,
            validation_data = (X_val, y_val_onehot),
            batch_size = batch_size,
            epochs = epochs,
            callbacks=[scal_checkpoint, early_stop],
            verbose=0
        )

        # evaluate model- load the saved model
        scal_model_saved = tf.keras.models.load_model(str(scal_model_name)+'.hdf5')
        _, acc = scal_model_saved.evaluate(X_test, y_test_onehot, verbose=0)
        print('> %.3f' % (acc * 100.0))
        
        
        y_pred = scal_model_saved.predict(X_test)
        fpr, tpr, thresholds = roc_curve(y_test, y_pred[:, 1])
        roc_auc = auc(fpr, tpr)
        results.append([acc, roc_auc])
        
        histories.append(history)
    res = pd.DataFrame( results, columns= ['accuracy', "roc_auc"])
    return histories, res

In [ ]:
def summarize_diagnostics(histories, title, path_save_plot_prefix, img_dpi=300 ):
    fig, (ax1, ax2) = plt.subplots(1,2, figsize=(14,7))
    for i in range(len(histories)):
        ax1.plot(histories[i].history['loss'], color='blue',  label='Training')
        ax1.plot(histories[i].history['val_loss'], color='orange', label='Validation')
    
        ax2.plot(histories[i].history['accuracy'], color='blue', label='Training')
        ax2.plot(histories[i].history['val_accuracy'], color='orange', label='Validation')
    
    ax1.set_title("Loss")
    ax1.set_ylabel("Loss")
    ax1.set_xlabel("Epochs")
    ax1.legend(frameon=False, labels = ["Training", "Validation"])
    
    ax2.set_title("Accuracy")
    ax2.set_ylabel("Accuracy")
    ax2.set_xlabel("Epochs")
    ax2.legend(frameon=False, labels = ["Training", "Validation"])

    fig.suptitle(title);
    fig.savefig(path_save_plot_prefix.parent.joinpath(os.path.basename(path_save_plot_prefix)+"_Model-perform.png"), bbox_inches='tight', dpi = img_dpi)
    
    return 


# Set paths

## Results paths

In [ ]:
path_res = Path(os.path.abspath("../demo_output/coloc_aggr_mt_skfold/"))
print(path_res)
path_res.is_dir()

In [ ]:
path_plots = path_res.joinpath("plots")
print(path_plots)
path_plots.is_dir()

In [ ]:
path_models = path_res.joinpath("saved_models")
print(path_models)
path_models.is_dir()

In [ ]:
if not path_res.is_dir():
    path_res.mkdir(parents=True, exist_ok=True)

if not path_plots.is_dir():
    path_plots.mkdir(parents=True, exist_ok=True)
    
if not path_models.exists():
    path_models.mkdir(parents=True, exist_ok=True)

# Read data

In [ ]:
data_file = Path(os.path.abspath("../demo_data/coloc_tab_demo_data.csv"))

#### Load the data

In [ ]:
df, experiment_cols, all_feats = load_data(data_file)

In [ ]:
print(df.shape)
display(df.head(2))
list(df.columns)
print( len(df.columns))
len(all_feats)

#### Update labels

In [ ]:
df.label.value_counts()

In [ ]:
df.label_name.value_counts()

In [ ]:
df.loc[(df.label_name=='Oligomycin') | (df.label_name=='rotenone'), "label_name"] = 'mito_toxicity'
df.loc[(df.label_name=='3xSNCA') | (df.label_name=='oligomer'), "label_name"] = 'aggregation'

In [ ]:
df.label_name.value_counts()

In [ ]:
#indicates disease
df.loc[df['label_name']=="aggregation","label"] = 0 
df.loc[df['label_name']=="mito_toxicity","label"] = 1
df.loc[df['label_name']=="control","label"] = 2

label_dict = {
    0: "Aggregation",
    1: "Mitochondrial toxicity",
    2: "control"
}
#label_dict[0]

In [ ]:
df.label.value_counts()

In [ ]:
#Total n of controls and diseases 
display_side_by_side(
    pd.DataFrame(df.label_name.value_counts()).rename(columns={"label_name":'label_count'}), 
    pd.DataFrame(np.round(df.label_name.value_counts()/len(df)*100, decimals=2)).rename(columns={"label_name":'percent'})
)

In [ ]:
df["PlateID"] = df.PlateID.astype('category')
df["label_name"] = df.label_name.astype('category')

In [ ]:
label_cols = ['label', 'label_name']

In [ ]:
df["PlateID"] = df.PlateID.astype('category')

# Evaluate the model : Dense Neural Network 

In [ ]:
path_prefix_model = path_models.joinpath("mixed-plates_coloc_aggr_mitoTox_")


In [ ]:
#Evaluate the model
histories, res = evaluate_model(data=df, all_features=all_feats, n_folds=10, variance_threshold=50, fname_prefix_save_models= path_prefix_model )

In [ ]:
print(res)

In [ ]:
#saving the accuracy scores 
res.to_pickle(path_models.joinpath("mixed-plates_coloc_aggr_mitoTox_accuracy_scores.pkl"))


# Plot model histories and scores

In [ ]:
plot_title = ("Coloc - Predicting diseases across plates, Aggregation vs mitotoxicity"
                            "Power transform per control within plate")
path_save_plot = path_plots.joinpath(os.path.basename(path_prefix_model))
summarize_diagnostics(histories, title=plot_title, path_save_plot_prefix = path_save_plot)

In [ ]:
def summarize_scores(scores, title ):
    #print summary
    print('Accuracy: mean=%.3f std=%.3f, n=%d' % (np.mean(scores)*100, np.std(scores)*100, len(scores)))
    # box and whisker plots of results
    plt.boxplot(scores)
    plt.suptitle(title)
    plt.show()
    return

In [ ]:
plot_title = ("Coloc - Predicting diseases across plates, Aggregation vs mitotoxicity"
                            "\n Accuracy")
summarize_scores(res.accuracy.values, plot_title )

In [ ]:
plot_title = ("Coloc - Predicting diseases across plates, Aggregation vs mitotoxicity"
                              "\n AUC ( Area under the ROC Curve)")
summarize_scores(res.roc_auc.values, plot_title )